<a href="https://colab.research.google.com/github/shubhanginimbalkar/AutoML_Project/blob/main/TPOT_Proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tpot


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.0 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=3287a7649562bb054de59953245ebfc93e82a0b52d5146d3770f70b2a37bbaa4
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [ ]:
# Import necessary libraries
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import time

# Load your dataset and handle data type warnings
data = pd.read_csv('/content/HomeC.csv', low_memory=False)

# Inspect column names to verify correct names
print(data.columns)

# Select relevant columns from your dataset (adjust based on the printed names)
selected_columns = [
    'time_use [kW]', 'gen [kW]', 'House overall [kW]', 'Dishwasher [kW]',
    'Furnace 1 [kW]', 'Furnace 2 [kW]', 'Home office [kW]', 'Fridge [kW]',
    'Wine cellar [kW]', 'Garage door [kW]', 'Kitchen 12 [kW]', 'Kitchen 14 [kW]',
    'Kitchen 38 [kW]', 'Barn [kW]', 'Well [kW]', 'Microwave [kW]',
    'Living room [kW]', 'Solar [kW]', 'temperature', 'icon', 'humidity',
    'visibility', 'summary', 'apparentTemperature', 'pressure', 'windSpeed',
    'cloudCover', 'windBearing', 'precipIntensity', 'dewPoint', 'precipProbability'
]

# Double-check the column names in your dataset and adjust if needed
X = data[selected_columns]
y = data['time_use [kW]']  # Adjust based on actual column name


Index(['time', 'use [kW]', 'gen [kW]', 'House overall [kW]', 'Dishwasher [kW]',
       'Furnace 1 [kW]', 'Furnace 2 [kW]', 'Home office [kW]', 'Fridge [kW]',
       'Wine cellar [kW]', 'Garage door [kW]', 'Kitchen 12 [kW]',
       'Kitchen 14 [kW]', 'Kitchen 38 [kW]', 'Barn [kW]', 'Well [kW]',
       'Microwave [kW]', 'Living room [kW]', 'Solar [kW]', 'temperature',
       'icon', 'humidity', 'visibility', 'summary', 'apparentTemperature',
       'pressure', 'windSpeed', 'cloudCover', 'windBearing', 'precipIntensity',
       'dewPoint', 'precipProbability'],
      dtype='object')


KeyError: "['time_use [kW]'] not in index"

In [ ]:
# Import necessary libraries
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import time

# Load the dataset
data = pd.read_csv('/content/HomeC.csv', low_memory=False)

# Drop the categorical columns 'icon', 'summary', and 'cloudCover'
data = data.drop(['icon', 'summary', 'cloudCover'], axis=1)

# Convert all columns to numeric, coercing errors (invalid parsing will be set as NaN)
data = data.apply(pd.to_numeric, errors='coerce')

# Handle missing values by imputing with the median
data.fillna(data.median(), inplace=True)

# Select the features (X) and target (y)
selected_columns = ['gen [kW]', 'House overall [kW]', 'Dishwasher [kW]',
                    'Furnace 1 [kW]', 'Furnace 2 [kW]', 'Home office [kW]',
                    'Fridge [kW]', 'Wine cellar [kW]', 'Garage door [kW]',
                    'Kitchen 12 [kW]', 'Kitchen 14 [kW]', 'Kitchen 38 [kW]',
                    'Barn [kW]', 'Well [kW]', 'Microwave [kW]', 'Living room [kW]',
                    'Solar [kW]', 'temperature', 'humidity', 'visibility',
                    'apparentTemperature', 'pressure', 'windSpeed',
                    'windBearing', 'precipIntensity', 'dewPoint', 'precipProbability']

X = data[selected_columns]
y = data['use [kW]']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TPOT with reduced parameters: limited pipelines and generations to reduce run-time
tpot = TPOTRegressor(generations=3, population_size=10, verbosity=2, random_state=42)

# Fit the model and track execution time
start_time = time.time()
tpot.fit(X_train, y_train)
end_time = time.time()

# Calculate and print execution time
execution_time = end_time - start_time
print(f"TPOT execution time: {execution_time:.2f} seconds")

# Get the top 5 models based on R² scores
results = []
for pipeline_str, pipeline_info in tpot.evaluated_individuals_.items():
    mse = -pipeline_info['internal_cv_score']  # Get negative MSE and convert to MSE
    variance = np.var(y_train)
    r2_score = 1 - (mse / variance)
    results.append((pipeline_str, r2_score, pipeline_info))

# Sort the results based on R²
results.sort(key=lambda x: x[1], reverse=True)

# Print the top 5 models with their R² scores and execution time
print("\nTop 5 Models by R²:")
for model, r2, pipeline_info in results[:5]:
    print(f'Model: {model}, R²: {r2:.4f}')


Optimization Progress:   0%|          | 0/40 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -4.8731774505502776e-11

Generation 2 - Current best internal CV score: -4.8731774505502776e-11

Generation 3 - Current best internal CV score: -4.8731774505502776e-11

Best pipeline: RidgeCV(input_matrix)
TPOT execution time: 6781.27 seconds

Top 5 Models by R²:
Model: RidgeCV(input_matrix), R²: 1.0000
Model: ElasticNetCV(input_matrix, ElasticNetCV__l1_ratio=1.0, ElasticNetCV__tol=0.1), R²: 1.0000
Model: ElasticNetCV(input_matrix, ElasticNetCV__l1_ratio=1.0, ElasticNetCV__tol=0.01), R²: 1.0000
Model: DecisionTreeRegressor(input_matrix, DecisionTreeRegressor__max_depth=8, DecisionTreeRegressor__min_samples_leaf=17, DecisionTreeRegressor__min_samples_split=5), R²: 0.9999
Model: DecisionTreeRegressor(MinMaxScaler(input_matrix), DecisionTreeRegressor__max_depth=7, DecisionTreeRegressor__min_samples_leaf=3, DecisionTreeRegressor__min_samples_split=4), R²: 0.9998


In [ ]:
import time
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Assuming 'X' is your feature set and 'y' is the target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to time model training
def time_model(model, X_train, y_train):
    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()
    return end_time - start_time

# RidgeCV
ridge_model = RidgeCV()
ridge_time = time_model(ridge_model, X_train, y_train)
print(f"RidgeCV execution time: {ridge_time:.2f} seconds")

# ElasticNetCV with l1_ratio=1.0 and tol=0.1
elasticnet_model1 = ElasticNetCV(l1_ratio=1.0, tol=0.1)
elasticnet_time1 = time_model(elasticnet_model1, X_train, y_train)
print(f"ElasticNetCV (l1_ratio=1.0, tol=0.1) execution time: {elasticnet_time1:.2f} seconds")

# ElasticNetCV with l1_ratio=1.0 and tol=0.01
elasticnet_model2 = ElasticNetCV(l1_ratio=1.0, tol=0.01)
elasticnet_time2 = time_model(elasticnet_model2, X_train, y_train)
print(f"ElasticNetCV (l1_ratio=1.0, tol=0.01) execution time: {elasticnet_time2:.2f} seconds")

# DecisionTreeRegressor with max_depth=8, min_samples_leaf=17, min_samples_split=5
decision_tree_model1 = DecisionTreeRegressor(max_depth=8, min_samples_leaf=17, min_samples_split=5)
decision_tree_time1 = time_model(decision_tree_model1, X_train, y_train)
print(f"DecisionTreeRegressor (max_depth=8, min_samples_leaf=17, min_samples_split=5) execution time: {decision_tree_time1:.2f} seconds")

# DecisionTreeRegressor with MinMaxScaler and max_depth=7, min_samples_leaf=3, min_samples_split=4
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
decision_tree_model2 = DecisionTreeRegressor(max_depth=7, min_samples_leaf=3, min_samples_split=4)
decision_tree_time2 = time_model(decision_tree_model2, X_train_scaled, y_train)
print(f"DecisionTreeRegressor with MinMaxScaler (max_depth=7, min_samples_leaf=3, min_samples_split=4) execution time: {decision_tree_time2:.2f} seconds")


RidgeCV execution time: 2.95 seconds
ElasticNetCV (l1_ratio=1.0, tol=0.1) execution time: 4.30 seconds
ElasticNetCV (l1_ratio=1.0, tol=0.01) execution time: 3.69 seconds
DecisionTreeRegressor (max_depth=8, min_samples_leaf=17, min_samples_split=5) execution time: 8.58 seconds
DecisionTreeRegressor with MinMaxScaler (max_depth=7, min_samples_leaf=3, min_samples_split=4) execution time: 6.07 seconds
